# Objectif : 
Ce notebook a pour but de **contrôler la correcte applicaiton des règles métiers** sur les jeux de données fournis par le courtier dans le cadre du reporting mensuel.

# Datasets :
- Contrats
- Quittances
- Sinistres
- Tarifs

# Étapes :
1. Contrôle des doublons
2. Imaginer les erreurs potentielles / Contrôle de cohérence
3. Contrôle de l'application du tarif
4. Contrôle des règles de gestion
5. Listing des anomalies

In [1]:
import os
import re
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("../scripts")
from checker import * 
from utils import * 

In [2]:
# Dossier contenant les données nettoyées
folder_path = "../data/processed"

# Mapping entre parties du nom de fichier et noms de variables souhaités
file_mapping = {
    "contrats": "df_contrats",
    "quittances": "df_quittances",
    "sinistres": "df_sinistres",
    "tarif": "df_tarifs"
}

# Chargement des fichiers Parquet
dataset = load_files_parquet(folder_path=folder_path, file_mapping=file_mapping)  # type: ignore

# Unpack
df_contrats = dataset["df_contrats"]
df_quittances = dataset["df_quittances"]
df_sinistres = dataset["df_sinistres"]
df_tarifs = dataset["df_tarifs"]

# Afficher l'ensemble des colonnes sans ellipses
pd.set_option('display.max_columns', None)

✅ Chargé : df_sinistres.parquet --> dataset['df_sinistres']
✅ Chargé : df_tarifs.parquet --> dataset['df_tarifs']
✅ Chargé : df_quittances.parquet --> dataset['df_quittances']
✅ Chargé : df_contrats.parquet --> dataset['df_contrats']


# 1. Contrôle des doublons / Clés primaires / Formats ID


1. Contrôle de la bijectivité de `coverRef` et `coverId`
2. Contrôle des doublons
3. Contrôle de l'uniformité des formats d'ID

In [3]:
#Contrôle relation 1:1 coverID / coverRef
df_anomalies_coverId = check_bijectivity_between_columns( # type: ignore
    df_contrats, 
    col_a="coverId", 
    col_b="coverRef", 
    anomaly_label="coverId_multiple_coverRef"
)

# Export si anomalie détectée
if not df_anomalies_coverId.empty:
    df_anomalies_coverId.to_csv("../outputs/anomalies_coverID_multiple_coverRef.csv", index=False)


🔍 Vérification de la bijectivité entre coverId et coverRef...
Nb de coverId associés à plusieurs coverRef : 1
⚠️ La relation coverId → coverRef n'est PAS bijective.
Nb de coverRef associés à plusieurs coverId : 0
✅ La relation coverRef → coverId est bijective (1:1).


On en déduit les clés : 
- Contrats : 
    - PK : `coverRef` (référence de contrat d'un animal assuré)
- Quittances : 
    - PK : `receiptId` (paiement mensuel d'un contrat)
    - FK : `df_contrats`.`coverRef`
- Sinistres : 
    - PK : combinaison (`incidentId`, `actId`) 
        - *NB : hypothèse - séquence sur les actesId*
    - FK : `df_contrats`.`coverRef`
- Tarifs : 
    - PK : Incorrecte / combinaison (`animal`, `age`, `couverture`)
        - *NB : hypothèse - clé étrangère manquante, ou PetRace*

In [4]:
primary_keys = {
    "df_contrats": ["coverId"],
    "df_quittances": ["receiptId"],
    "df_sinistres": ["actId", "incidentId"],
    "df_tarifs": ["animal", "age", "couverture"]
}

duplicates_report = check_all_duplicates_with_keys(dataset, primary_keys=primary_keys) # type: ignore


🔍 df_sinistres : détection des doublons
➡️ 1 doublons complets détectés.


,actDescription,claimPaid,notificationDate,guarantee,claimOutstanding,actDate,actCategory,coverRef,coverId,actType,actId,incidentId,actValue,incidentDate
130,[],12.24,2021-12-22 14:26:01.956301,0,0,2021-12-20,ACCIDENT,sub_1K2z9JF5i98BakySUuC2Y9yl-250268502090843,Sub_DLM_1552191-0,PHAR,C_qrtodl08_4,C_qrtodl08,15.3,2021-12-17
131,[],12.24,2021-12-22 14:26:01.956301,0,0,2021-12-20,ACCIDENT,sub_1K2z9JF5i98BakySUuC2Y9yl-250268502090843,Sub_DLM_1552191-0,PHAR,C_qrtodl08_4,C_qrtodl08,15.3,2021-12-17


⚠️ Affichage limité à 10 lignes pour les doublons complets.

➡️ 1 violations de la clé primaire ['actId', 'incidentId'].


,actDescription,claimPaid,notificationDate,guarantee,claimOutstanding,actDate,actCategory,coverRef,coverId,actType,actId,incidentId,actValue,incidentDate
130,[],12.24,2021-12-22 14:26:01.956301,0,0,2021-12-20,ACCIDENT,sub_1K2z9JF5i98BakySUuC2Y9yl-250268502090843,Sub_DLM_1552191-0,PHAR,C_qrtodl08_4,C_qrtodl08,15.3,2021-12-17
131,[],12.24,2021-12-22 14:26:01.956301,0,0,2021-12-20,ACCIDENT,sub_1K2z9JF5i98BakySUuC2Y9yl-250268502090843,Sub_DLM_1552191-0,PHAR,C_qrtodl08_4,C_qrtodl08,15.3,2021-12-17


⚠️ Affichage limité à 10 lignes pour les violations de clé primaire.

------------------------------------------------------------

🔍 df_tarifs : détection des doublons
➡️ 0 doublons complets détectés.
✅ Aucun doublon complet détecté.

➡️ 702 violations de la clé primaire ['animal', 'age', 'couverture'].


,animal,age,couverture,taux,healthLimit,franchise,healthHthcMonthly
1,cat,0,Accident,0.6,1000,0,6.78
19,cat,0,Accident,0.7,1000,0,8.02
37,cat,0,Accident,0.8,1000,0,9.28
55,cat,0,Accident,0.9,1000,0,10.55
73,cat,0,Accident,1.0,1000,0,11.83
91,cat,0,Accident,0.6,1200,0,7.56
109,cat,0,Accident,0.7,1200,0,8.99
127,cat,0,Accident,0.8,1200,0,10.43
145,cat,0,Accident,0.9,1200,0,11.90
163,cat,0,Accident,1.0,1200,0,13.38


⚠️ Affichage limité à 10 lignes pour les violations de clé primaire.

------------------------------------------------------------

🔍 df_quittances : détection des doublons
➡️ 0 doublons complets détectés.
✅ Aucun doublon complet détecté.

➡️ 0 violations de la clé primaire ['receiptId'].
✅ Aucun doublon sur la clé primaire.

------------------------------------------------------------

🔍 df_contrats : détection des doublons
➡️ 0 doublons complets détectés.
✅ Aucun doublon complet détecté.

➡️ 1 violations de la clé primaire ['coverId'].


,healthPremiumInclTax,preventionTax,healthBrokerFee,preventionPremiumInclTax,coverRef,petRace,liabilityCovered,coverEndDate,coverRate,petSick,petUuidType,petUuid,petType,petOwnerBirthday,coverId,healthLimit,preventionBrokerFee,preventionHthc,petOwnerName,coverStartDate,preventionLimit,healthHthc,petSex,healthTax,petBirthday,petName,petOwnerAddress,petOldDiseases,deductibleLimit,customerId
669,193.45,8.88,32.04,108.0,sub_K0aryWqH87FdhH-250268502141316,ragdoll,False,2022-09-08,0.8,healthy,chip,250268502141316,cat,1960-01-31,Sub_1817512-0,1500,-0.84,99.96,xxx,2021-09-08,100,144.00,MALE,17.410549,2021-04-01,Cléopâtre,xxxFrance,[],0,Cus_1168464
800,374.90,8.88,116.52,108.0,sub_K6wP0rlfZtLZBi-250269810316536,mixed_breed,False,2022-08-26,0.6,healthy,chip,250269810316536,dog,1973-02-07,Sub_1817512-0,1500,-0.84,99.96,xxx,2021-08-26,100,224.64,FEMALE,33.741099,2013-10-01,Cléopâtre,xxxFrance,[],0,Cus_1682219


⚠️ Affichage limité à 10 lignes pour les violations de clé primaire.

------------------------------------------------------------



In [5]:
anomalies_global = scan_all_datasets_for_id_anomalies(dataset) # type: ignore


🔍 Analyse du dataset : df_sinistres

🔍 Analyse du dataset : df_tarifs

🔍 Analyse du dataset : df_quittances

🔍 Analyse du dataset : df_contrats

🆔 Analyse de la colonne 'petUuid'
🔍 Nombre de valeurs entièrement alphabétiques : 7
⚠️ Exemples de valeurs suspectes :
['chip']

✅ Anomalies globales exportées vers 'anomalies_ids_format_global.csv'


In [6]:
#contrôle des trous de séquence (acteId) sur df_sinistres

# 2. Contrôles de cohérence 

In [7]:
run_all_consistency_checks(df_contrats, df_quittances, df_sinistres)# type: ignore


========== 📄 LANCEMENT DES CONTRÔLES DE COHÉRENCE ==========

🔹 1. Contrats sans quittance
[check_contracts_have_quittance] Nombre de contrats sans quittance détectés : 2
[check_contracts_have_quittance] Nombre de coverRef distincts concernés : 2
[check_contracts_have_quittance] Montant total des primes concernées : 383.97 €
[check_contracts_have_quittance] coverRefs concernés : ['sub_JPvnlDW8nBmHjP-250269608337058', 'sub_K0aryWqH87FdhH-250268502141316']
------------------------------------------------------------

🔹 2. Quittances sans contrat
[check_quittance_have_matching_contracts] Nombre de Quittances en Anomalies détectées : 10
[check_quittance_have_matching_contracts] Nombre de coverRef distincts sans contrat : 7
[check_quittance_have_matching_contracts] Total Primes des quittances concernées : 235.90 €
[check_quittance_have_matching_contracts] coverRef concernés : ['sub_1JrMc4F5i98BakyS905SV9sR-250268732548727', 'sub_1JnK6fF5i98BakySZFBryTyk-25026872004422', 'sub_JzTgPvMcxdPfrx

# Règles métiers

In [8]:
#Durée et renouvellement
check_contract_duration(df_contrats)


[check_contract_duration] Nombre de contrats avec durée ≠ 1 an : 3
[check_contract_duration] Nombre de coverRef concernés : 3
[check_contract_duration] coverRefs concernés : ['sub_KCBrp1lZVfGMEg-250268600336306', 'sub_1JeygtF5i98BakySXfQNEGzc-250269300150647', 'sub_1K4oMKF5i98BakySZtKVfBoK-250269590673101']


,healthPremiumInclTax,preventionTax,healthBrokerFee,preventionPremiumInclTax,coverRef,petRace,liabilityCovered,coverEndDate,coverRate,petSick,petUuidType,petUuid,petType,petOwnerBirthday,coverId,healthLimit,preventionBrokerFee,preventionHthc,petOwnerName,coverStartDate,preventionLimit,healthHthc,petSex,healthTax,petBirthday,petName,petOwnerAddress,petOldDiseases,deductibleLimit,customerId,anomaly_type
970,265.98,8.88,52.68,108.0,sub_KCBrp1lZVfGMEg-250268600336306,berger_belge_malinois,False,2021-10-28,1.0,healthy,chip,250268600336306,dog,1991-05-24,Sub_1281984-0,1000,-0.84,99.96,xxx,2021-09-09,100,189.36,MALE,23.938022,2021-06-01,Saiko,xxxFrance,[],0,Cus_1884143,contract_duration_not_1_year
1236,350.77,8.88,71.28,108.0,sub_1JeygtF5i98BakySXfQNEGzc-250269300150647,berger_australien,False,2022-11-29,1.0,healthy,chip,250269300150647,dog,1955-06-27,Sub_1648341-0,1500,-0.84,99.96,xxx,2021-09-29,100,247.92,MALE,31.569231,2021-03-01,SIRIUS,xxxFrance,[],0,Cus_1937163,contract_duration_not_1_year
3046,208.51,8.88,41.30,108.0,sub_1K4oMKF5i98BakySZtKVfBoK-250269590673101,pinscher,False,2021-09-12,0.8,healthy,chip,250269590673101,dog,1964-11-26,Sub_DLM_1372702-0,1000,-0.84,99.96,xxx,2021-09-12,100,148.44,MALE,18.765495,2021-09-01,Sushy,xxxFrance,[],0,Cus_1195092,contract_duration_not_1_year


In [9]:
#eligibilité des animaux (puce, tatouage, age)
df_anomalies, recap = check_eligibilite_animaux(df_contrats)
display(recap)  


[check_eligibilite_animaux] Nombre d’animaux non éligibles : 255
[check_eligibilite_animaux] Nombre de customerId distincts concernés : 255
[check_eligibilite_animaux] coverRefs concernés : ['sub_JSTA6dBG7fvUQA-250268731585060', 'sub_JSWB8mIMvWdQ9s-250268743652706', 'sub_JSXmjsVvWxRMgP-250268731492607', 'sub_JVAKExJcCr9pWZ-250268731355558', 'sub_JY8TI8whGXhEeH-900108001316034', 'sub_JZD3fXFI7akdCO-250269590327490', 'sub_JZIK5X8JtepTua-234UPC', 'sub_JcwsPDyyEv1LJO-250268500839026', 'sub_JdPmL0UK6WR97r-119KjX', 'sub_JdgpBYuQP6JDKx-250268712392556', 'sub_JeybdRY9VKm5aZ-172CDW', 'sub_JfGhKzgqe4pNsk-250269810304389', 'sub_Ji8sUyCTZ6vdka-250269606618456', 'sub_JiJD65yZROg9rB-250268731356504', 'sub_JiZw4oaFlapLMj-LOOF 2020.14896', 'sub_Jis4zaGJqpEXGn-250268730221179', 'sub_Jj2ml5qe17iivA-250268501978116', 'sub_JjOq2wR3Uj8qLI-250269802663558', 'sub_JkodwW3RpJq8kO-250268500857416', 'sub_Jl9jikuOYl6LHU-170SRE', 'sub_JltDisowzfrO3B-250269810084922', 'sub_Jlzbd3nWxKx4dD-250268500600909', 'sub_JmcW

,cause,nombre_d_animaux
0,age_out_of_bounds,249
1,invalid_uuid_format,13


In [10]:
# application du tarif prévention
df_anomalies_prevention = check_tarif_prevention(df_contrats)
display(df_anomalies_prevention.head())

[check_tarif_prevention] Nombre d’anomalies tarifaires Prévention : 0
[check_tarif_prevention] Nombre de customerId concernés : 0
[check_tarif_prevention] coverRefs concernés : []


""


In [11]:
#application du tarif Health
df_anomalies_tarif_health = check_tarif_health(df_contrats, df_tarifs)

df_anomalies_tarif_health.head()

[check_tarif_health] Contrôle non réalisable : colonnes manquantes dans df_tarifs -> {'PetRace', 'CoverId', 'CoverRef'}


""


In [12]:
#Check arithmétique df_quittances
df_incohérences = check_arithmetic_consistency_quittances(df_quittances).sort_values(by='diff', ascending=False)

if not df_incohérences.empty:
    print("Incohérences détectées :")
    display(df_incohérences)
else:
    print("Tout est cohérent ✅")

Incohérences détectées :


,coverRef,receiptId,healthPremiumInclTax,healthTax,healthBrokerFee,healthHthc,expected_premium,diff
6889,sub_Jd0fyG17zUxvpc-250269590326851,no_discount-in_1JhynxF5i98BakySnSvSv60B-250269...,19.99,3.580714,5.2250,30.98,39.785714,19.795714
1695,sub_K6wP0rlfZtLZBi-250269810316536,no_discount-2CCBD08A-13792-0,30.99,1.450879,2.6700,12.00,16.120879,14.869121
1696,sub_K6wP0rlfZtLZBi-250269810316536,no_discount-in_1JSiWFF5i98BakyS8pmBmyNm-250269...,30.99,1.450879,2.6700,12.00,16.120879,14.869121
1697,sub_K6wP0rlfZtLZBi-250269810316536,no_discount-in_1JdxIOF5i98BakyS1IeXgqE0-250269...,30.99,1.450879,2.6700,12.00,16.120879,14.869121
1698,sub_K6wP0rlfZtLZBi-250269810316536,no_discount-in_1Jopb8F5i98BakySgmuLPSk5-250269...,30.99,1.450879,2.6700,12.00,16.120879,14.869121
...,...,...,...,...,...,...,...,...
5041,sub_Jqq5yF3toTLv4a-941000024876724,no_discount-2CCBD08A-12193-0,21.99,1.886291,3.2925,15.78,20.958791,1.031209
5045,sub_Jqq5yF3toTLv4a-941000024876724,no_discount-in_1JkUG6F5i98BakySG07fpeSR-941000...,21.99,1.886291,3.2925,15.78,20.958791,1.031209
5059,sub_1K2CaBF5i98BakySBaqEb4L0-250268732430818,no_discount-2CCBD08A-10313-0,26.99,2.337527,5.6250,18.01,25.972527,1.017473
1219,sub_1K2XzbF5i98BakySC4yeK6JP-967000010048780,no_discount-2CCBD08A-10492-0,26.99,2.337527,5.6250,18.01,25.972527,1.017473


In [13]:
# contrôle contrats : décomposition prime calcul de la prime d'assurance au global (ttc, ht, prime courtier, distributeurs)
df_contrats_ko = check_arithmetic_consistency_contrats(df_contrats)

if not df_contrats_ko.empty:
    print("⚠️ Incohérences dans les montants TTC des contrats détectées :")
    display(df_contrats_ko)
else:
    print("✅ Tous les montants TTC des contrats sont cohérents.")

⚠️ Incohérences dans les montants TTC des contrats détectées :


,coverRef,healthPremiumInclTax,healthBrokerFee,healthHthc,healthTax,expected_premium,diff
1612,sub_1JlefZF5i98BakyS1smjanLb-250268600166865,511.16,95.28,278.88,37.004835,411.164835,99.995165


In [14]:
#Contrôles décaissements - plafond de remboursement (sinistres.claimPaid))

df_limits_violated = check_reimbursement_limits_with_contracts(df_sinistres, df_contrats)

if not df_limits_violated.empty:
    print("🚨 Dépassements de plafond détectés :")
    display(df_limits_violated)
else:
    print("✅ Tous les remboursements sont dans les plafonds définis.")


🚨 Dépassements de plafond détectés :


,coverRef,year,actCategory,totalClaimPaid,limit,overLimit
813,sub_JSQboCecJMykVb-642098100264850,2021.0,ACCIDENT,1305.2,1000.0,305.2
976,sub_JhP79RdcZ7katc-250269590456576,2021.0,PREVENTION,36.9,0.0,36.9
1562,sub_KE0ovKjw9L5aWM-250268743963582,2021.0,PREVENTION,136.4,100.0,36.4


In [15]:
#contrôle arithmétique - taux de remboursement
anomalies_taux = check_taux_remboursement(df_sinistres, df_contrats)
display(anomalies_taux.head())

[check_taux_remboursement] Nombre d’anomalies de remboursement : 78
[check_taux_remboursement] Nombre de coverRefs concernés : 44


,coverRef,actCategory,actValue,coverRate,claimPaid,expected_claimPaid,delta,anomaly_type
67,sub_JYlxIruefxZCri-250269590501363,MALADIE,7.8,1.0,0.0,7.80,-7.80,incorrect_reimbursement_rate
171,sub_1Js9vqF5i98BakyS3GgtDbU3-250269100287700,ACCIDENT,3.6,0.8,3.6,2.88,0.72,incorrect_reimbursement_rate
177,sub_JgQafZrqcZe9HP-250269100219109,ACCIDENT,44.0,0.8,44.0,35.20,8.80,incorrect_reimbursement_rate
178,sub_JgQafZrqcZe9HP-250269100219109,ACCIDENT,58.0,0.8,58.0,46.40,11.60,incorrect_reimbursement_rate
179,sub_JgQafZrqcZe9HP-250269100219109,ACCIDENT,13.0,0.8,13.0,10.40,2.60,incorrect_reimbursement_rate


In [16]:
#Contrôles décaissements - délai de carence (df_sinistres)
anomalies_carence = check_delai_carence(df_sinistres, df_contrats)
display(anomalies_carence.head())

[check_delai_carence] Nombre d’anomalies de carence détectées : 480
[check_delai_carence] Nombre de coverRefs concernés : 129


,coverRef,actCategory,actType,incidentDate,coverStartDate,carence_days,carence_end_date,days_before_eligibility,claimPaid,anomaly_type
39,sub_JpakMyv4nzqxvr-250269590444547,PREVENTION,VERM,2021-09-21,2021-11-07,0,2021-11-07,47.0,13.65,reimbursement_before_carence
40,sub_JpakMyv4nzqxvr-250269590444547,PREVENTION,VERM,2021-07-15,2021-11-07,0,2021-11-07,115.0,6.49,reimbursement_before_carence
41,sub_JpakMyv4nzqxvr-250269590444547,PREVENTION,VACC,2021-07-28,2021-11-07,0,2021-11-07,102.0,58.00,reimbursement_before_carence
49,sub_JOQycpLQ2NuWUR-250269608770991,MALADIE,PHAR,2021-05-28,2021-04-29,45,2021-06-13,16.0,11.65,reimbursement_before_carence
50,sub_JOQycpLQ2NuWUR-250269608770991,MALADIE,PHAR,2021-05-28,2021-04-29,45,2021-06-13,16.0,12.50,reimbursement_before_carence


In [17]:
df_anomalies_negatives = check_negative_values_on_all_datasets(dataset)# type: ignore



🔎 Contrôle des valeurs négatives dans 'df_sinistres'
⚠️  claimPaid contient 3 valeur(s) négative(s)
    Exemples : [-0.4, -54.0, -99.8]
⚠️  actValue contient 3 valeur(s) négative(s)
    Exemples : [-0.4, -54.0, -99.8]

🔎 Contrôle des valeurs négatives dans 'df_tarifs'
✅ Aucune valeur négative trouvée dans 'df_tarifs'

🔎 Contrôle des valeurs négatives dans 'df_quittances'
⚠️  healthPremiumInclTax contient 24 valeur(s) négative(s)
    Exemples : [-1.8, -1.8, -1.8, -1.8, -1.8]
⚠️  healthTax contient 24 valeur(s) négative(s)
    Exemples : [-0.157524725, -0.157524725, -0.157524725, -0.157524725, -0.157524725]
⚠️  healthBrokerFee contient 24 valeur(s) négative(s)
    Exemples : [-0.20075, -0.20075, -0.20075, -0.20075, -0.20075]
⚠️  preventionBrokerFee contient 6280 valeur(s) négative(s)
    Exemples : [-0.07, -0.07, -0.07, -0.07, -0.07]
⚠️  healthHthc contient 24 valeur(s) négative(s)
    Exemples : [-1.392, -1.392, -1.392, -1.392, -1.392]

🔎 Contrôle des valeurs négatives dans 'df_contrat

In [18]:
results_part1, anomalies_part1 = run_quality_pipeline_part1(df_contrats, df_quittances, df_tarifs)



========== 📋 LANCEMENT DU PIPELINE QUALITÉ CONTRATS — PARTIE 1 ==========

🔹 1. Durée des contrats — Vérifie que chaque contrat dure exactement 1 an
[check_contract_duration] Nombre de contrats avec durée ≠ 1 an : 3
[check_contract_duration] Nombre de coverRef concernés : 3
[check_contract_duration] coverRefs concernés : ['sub_KCBrp1lZVfGMEg-250268600336306', 'sub_1JeygtF5i98BakySXfQNEGzc-250269300150647', 'sub_1K4oMKF5i98BakySZtKVfBoK-250269590673101']

🔹 2. Éligibilité des animaux — Vérifie l'âge, la présence et le format du numéro d'identification
[check_eligibilite_animaux] Nombre d’animaux non éligibles : 255
[check_eligibilite_animaux] Nombre de customerId distincts concernés : 255
[check_eligibilite_animaux] coverRefs concernés : ['sub_JSTA6dBG7fvUQA-250268731585060', 'sub_JSWB8mIMvWdQ9s-250268743652706', 'sub_JSXmjsVvWxRMgP-250268731492607', 'sub_JVAKExJcCr9pWZ-250268731355558', 'sub_JY8TI8whGXhEeH-900108001316034', 'sub_JZD3fXFI7akdCO-250269590327490', 'sub_JZIK5X8JtepTua-234

In [19]:
results, anomalies = run_quality_pipeline_part2(df_contrats, df_quittances, df_sinistres, results_part1, anomalies_part1)




========== 📋 SUITE DU PIPELINE QUALITÉ CONTRATS — PARTIE 2 ==========

🔹 7. Dépassement des plafonds — Vérifie que les remboursements ne dépassent pas les limites prévues au contrat
[check_reimbursement_limits_with_contracts] ⚠️ 3 dépassement(s) détecté(s)

🔹 8. Taux de remboursement — Vérifie que claimPaid = actValue * coverRate pour certaines catégories d’actes


[check_taux_remboursement] Nombre d’anomalies de remboursement : 78
[check_taux_remboursement] Nombre de coverRefs concernés : 44

🔹 9. Délai de carence — Vérifie que les remboursements ont lieu après les délais de carence contractuels
[check_delai_carence] Nombre d’anomalies de carence détectées : 480
[check_delai_carence] Nombre de coverRefs concernés : 129

🔹 10. Valeurs négatives — Vérifie qu’aucune valeur numérique n’est strictement négative dans les jeux de données

🔎 Contrôle des valeurs négatives dans 'df_contrats'
⚠️  preventionBrokerFee contient 2149 valeur(s) négative(s)
    Exemples : [-0.84, -0.84, -0.84, -0.84, -0.77]

🔎 Contrôle des valeurs négatives dans 'df_quittances'
⚠️  healthPremiumInclTax contient 24 valeur(s) négative(s)
    Exemples : [-1.8, -1.8, -1.8, -1.8, -1.8]
⚠️  healthTax contient 24 valeur(s) négative(s)
    Exemples : [-0.157524725, -0.157524725, -0.157524725, -0.157524725, -0.157524725]
⚠️  healthBrokerFee contient 24 valeur(s) négative(s)
    Exemples

In [20]:
export_pipeline_anomalies(anomalies)


📤 Export des anomalies détectées :
🟠 Exporté : ../outputs/anomalies_contract_duration.csv
🟠 Exporté : ../outputs/anomalies_eligibilite_animaux.csv
🟢 Aucun problème détecté → anomalies_tarif_prevention.csv
🟢 Aucun problème détecté → anomalies_tarif_health.csv
🟠 Exporté : ../outputs/anomalies_arithmetique_contrats.csv
🟠 Exporté : ../outputs/anomalies_arithmetique_quittances.csv
🟠 Exporté : ../outputs/anomalies_plafond_remboursement.csv
🟠 Exporté : ../outputs/anomalies_taux_remboursement.csv
🟠 Exporté : ../outputs/anomalies_delai_carence.csv
🟠 Exporté : ../outputs/anomalies_valeurs_negatives.csv

========== ✅ EXPORT TERMINÉ ==========


# Synthèse des anomalies

In [21]:
df_contrats.head()

,healthPremiumInclTax,preventionTax,healthBrokerFee,preventionPremiumInclTax,coverRef,petRace,liabilityCovered,coverEndDate,coverRate,petSick,petUuidType,petUuid,petType,petOwnerBirthday,coverId,healthLimit,preventionBrokerFee,preventionHthc,petOwnerName,coverStartDate,preventionLimit,healthHthc,petSex,healthTax,petBirthday,petName,petOwnerAddress,petOldDiseases,deductibleLimit,customerId
0,272.53,0.00,55.88,0.00,sub_JO4FupTmKndXKI-250269608454258,bouvier_bernois,False,2022-04-28,0.8,healthy,chip,250269608454258,dog,1989-07-27,Sub_1880770,1500,0.00,0.00,xxx,2021-04-28,0,192.12,FEMALE,24.527473,2020-01-29,Roquette,xxxFrance,[],0,Cus_1813539
1,272.53,4.88,55.88,59.54,sub_JOIIjjikIBaeyo-250268502022433,berger_creole,False,2022-04-29,0.8,healthy,chip,250268502022433,dog,1987-09-02,Sub_1643327,1500,4.62,50.04,xxx,2021-04-29,50,192.12,FEMALE,24.527473,2020-09-20,Soba,xxxFrance,[],0,Cus_1091499
2,238.71,0.00,31.35,0.00,sub_JOKXQzz6v8COZC-250269300000660,norvegien,False,2022-04-29,1.0,not_healthy,chip,250269300000660,cat,1987-03-11,Sub_1323061,1500,0.00,0.00,xxx,2021-04-29,0,185.88,MALE,21.484286,2019-10-28,PRINCE LASHA,xxxFrance,['Cystite'],0,Cus_1867875
3,238.71,0.00,31.35,0.00,sub_JOQycpLQ2NuWUR-250269608770991,somali,False,2022-04-29,1.0,not_healthy,chip,250269608770991,cat,1996-05-16,Sub_1286981,1500,0.00,0.00,xxx,2021-04-29,0,185.88,MALE,21.484286,2020-10-24,Rouflaquette,xxxFrance,['Autre'],0,Cus_1820361
4,350.77,4.92,71.28,60.00,sub_JOeRKHAuTCzVP7-250269608652865,golden-retriever,False,2022-04-30,1.0,healthy,chip,250269608652865,dog,1985-11-15,Sub_1121929,1500,5.04,50.04,xxx,2021-04-30,50,247.92,FEMALE,31.569231,2020-09-06,Rhapsodie,xxxFrance,[],0,Cus_1624175


In [22]:
df_recap_anomalies = export_all_anomalies_to_excel("../outputs", "../outputs/anomalies_report.xlsx")# type: ignore
display(df_recap_anomalies) 

✅ Exporté : anomalies_plafond_remboursement.csv → feuille 'plafond_remboursement'
✅ Exporté : anomalies_taux_remboursement.csv → feuille 'taux_remboursement'
✅ Exporté : anomalies_sinistres_sans_contrat.csv → feuille 'sinistres_sans_contrat'
✅ Exporté : anomalies_cleaning.csv → feuille 'cleaning'
✅ Exporté : anomalies_arithmetique_contrats.csv → feuille 'arithmetique_contrats'
✅ Exporté : anomalies_contract_duration.csv → feuille 'contract_duration'
✅ Exporté : anomalies_eligibilite_animaux.csv → feuille 'eligibilite_animaux'
✅ Exporté : anomalies_coverID_multiple_coverRef.csv → feuille 'coverID_multiple_coverRef'
✅ Exporté : anomalies_valeurs_negatives.csv → feuille 'valeurs_negatives'
✅ Exporté : anomalies_contrats_sans_quittance.csv → feuille 'contrats_sans_quittance'
✅ Exporté : anomalies_arithmetique_quittances.csv → feuille 'arithmetique_quittances'
✅ Exporté : anomalies_quittances_sans_contrat.csv → feuille 'quittances_sans_contrat'
✅ Exporté : anomalies_ids_format_global.csv → 

,Type d'anomalie,Nb de lignes,Nb de contrats uniques,Montant total concerné (€),Agrégat concerné
0,plafond_remboursement,3,3,1478.5,totalClaimPaid
1,taux_remboursement,78,44,5916.24,claimPaid
2,sinistres_sans_contrat,6,2,197.5,claimPaid
3,cleaning,172,-,-,-
4,arithmetique_contrats,1,1,511.16,healthPremiumInclTax
5,contract_duration,3,3,825.26,healthPremiumInclTax
6,eligibilite_animaux,255,255,97354.28,healthPremiumInclTax
7,coverID_multiple_coverRef,2,2,568.35,healthPremiumInclTax
8,valeurs_negatives,8531,2158,-308.4,claimPaid
9,contrats_sans_quittance,2,2,383.97,healthPremiumInclTax


In [ ]:
# Copie du DataFrame sans les lignes où les infos sont absentes
df_bubble = df_recap_anomalies.dropna(subset=["Montant total concerné (€)", "Nb de contrats uniques"]).copy()

# Nettoyage de la colonne "Montant total concerné (€)"
df_bubble["Montant total concerné (€)"] = (
    df_bubble["Montant total concerné (€)"]
    .replace("-", np.nan)  # ou .replace("-", 0.0) si tu préfères
    .astype(float)
)

# Conversion des contrats uniques (si besoin)
df_bubble["Nb de contrats uniques"] = pd.to_numeric(df_bubble["Nb de contrats uniques"], errors="coerce").fillna(0).astype(int)

# Graphe
plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    df_bubble["Nb de contrats uniques"],
    df_bubble["Nb de lignes"],
    s=df_bubble["Montant total concerné (€)"].abs() / 10 + 50,  # taille des bulles
    c=df_bubble["Agrégat concerné"].astype("category").cat.codes,
    cmap="Set2",
    alpha=0.7,
    edgecolors="w",
    linewidth=0.5
)

# Étiquettes
for i, row in df_bubble.iterrows():
    plt.text(row["Nb de contrats uniques"]+2, row["Nb de lignes"], row["Type d'anomalie"], fontsize=8)

plt.xlabel("Nombre de contrats uniques")
plt.ylabel("Nombre de lignes")
plt.title("Gravité des anomalies (bubble chart)")
plt.grid(True)
plt.tight_layout()
plt.show()


ValueError: could not convert string to float: '-'

# Conclusion

Compte tenu du nombre d'anomalie relevées, une seconde itération avec le courtier après une première correction des données pourrait être bénéfique. 

En dehors des anomalies listées dans le `report.xlsx`, les points suivants sont notables : 

- df_tarifs : 
    - La Garantie Health couvre les soins de Maladie et d'Accident : or la table tarif ne contient qu'Accident dans la colonne couverture
    - Table inexploitable : pour un type d'animal, et un âge, plusieurs montant de prime, taux de couverte sont listés sans possibilité de distinguer la police associée (clé étrangère manquante, ou PetRace)
    
- df_quittance : 
    - Hypothèse : Guarantee = claimPaid + claimOutstanding. Guarantee est nul -> A VENTILER pour suivre le montant total restant

- df_contrats : 
    - eligibilité des animaux : absence d'information sur la formule pour choix de l'âge maximal couvert (*"7 ou 9 ans selon la formule choisie à l'adhésion"*)
    - reconduction / résiliation : sauf erreur, nous n'avons pas d'information sur la date de résiliation. Nous avons donc considéré que les contrats s'étaient tacitement reconduits. 

Next steps (lorsque tables contrats et tarifs propres): 
- reconciliation contrats / quittances (+ 1 quittance par mois où le contrat est actif)
- réconciliation tarifs Health (yc multi-contrat discounts)